In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
from torchvision import datasets,transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook as tqdm
from torchvision.utils import save_image
from torch.autograd import Variable

In [ ]:
def seed_everything(seed = 224):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()    

# 这里只是为了保证我们的可复现性



In [ ]:
BATCH_SIZE = 256
IMAGE_SIZE = 64
def tensorGenerater(path = '../input/all-dogs/',batch_size = BATCH_SIZE,ouputimg_size = IMAGE_SIZE,rotate_size = 10):
    train_transform = transforms.Compose([transforms.Resize(64),
                                          transforms.RandomHorizontalFlip(),
                                            transforms.RandomRotation(rotate_size), # 我们的目标size 是64 by 64 呀！
                                            transforms.CenterCrop(ouputimg_size),
                                            transforms.ToTensor(),
                                            transforms.Normalize(mean=[.5,.5,.5],
                                                                 std =[.5,.5,.5])]) #保证数字在-1，1之间
    traindata = datasets.ImageFolder(path,transform=train_transform)
    trainloader = torch.utils.data.DataLoader(traindata,batch_size,shuffle=True)
    return trainloader




In [ ]:
trainloader = tensorGenerater()

看看data

In [ ]:
imgs, label = iter(trainloader).next()
imgs.size(),label.size() # 一个batch 的data（tensor）


In [ ]:
img = imgs[0]

print('Min: ', imgs.min())
print('Max: ', imgs.max())

看看狗子

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  # unnormalize，从（-1，1） >>>>>>>> （0，1）
    plt.imshow(np.transpose(img, (1, 2, 0))) #因为numpy plot 要通道数在最后
    
fig = plt.figure(figsize=(25, 16))
for idx in range(64):
    ax = fig.add_subplot(8, 8, idx+1, xticks=[], yticks=[])
    imshow(imgs[idx])
    ax.set_title("{}".format(label[idx])) # 

这是时候就是model 的结构了，GAN 有两个部分Generator 和 Descriminator。先做Descriminator。

In [ ]:
def conv(inchanel,outchanel,kernelsize = 4,stride = 2,padding = 1 ,BN = True):
    layers = []
    conv_layer = nn.Conv2d(inchanel,outchanel,kernelsize,
                           stride=stride,padding= padding,bias=False)
    layers.append(conv_layer)
    if BN:
        layers.append(nn.BatchNorm2d(outchanel))
    
    return nn.Sequential(*layers)

class Des(nn.Module):
    def __init__(self,conv_dim = 32):
        super(Des,self).__init__()
        self.conv_dim = conv_dim
        
        # layers
        self.conv1 = conv(3,conv_dim * 2, 4)
        self.conv2 = conv(conv_dim * 2, conv_dim * 4, 4,BN=False)
        self.conv3 = conv(conv_dim * 4, conv_dim * 8, 4,BN=False)
        self.conv4 = conv(conv_dim * 8, conv_dim * 16,4)
        self.conv5 = nn.Conv2d(conv_dim * 16,1,4,1,0,bias=False) # 256 >>>1 , 4 by 4 >>>> 1 by 1
        
    def forward(self,x):
        x = F.leaky_relu(self.conv1(x),0.2) #64 * 32 * 32
        x = F.leaky_relu(self.conv2(x),0.2) # 128 * 16 * 16 
        x = F.leaky_relu(self.conv3(x),0.2) # 256 * 8 * 8 
        x = F.leaky_relu(self.conv4(x),0.2) # 512 * 4 * 4
        x = torch.sigmoid(self.conv5(x)).view(-1,1)
        
        return x
        

Generator

In [ ]:
def tconv(inchanel,outchanel,kernelsize = 4,stride = 2,padding = 1 ,BN = True):
    layers = []
    conv_layer = nn.ConvTranspose2d(inchanel,outchanel,kernelsize,
                                   stride=stride,padding=padding)
    layers.append(conv_layer)
    if BN:
        layers.append(nn.BatchNorm2d(outchanel))
    
    return nn.Sequential(*layers)


class Generator(nn.Module):
    def __init__(self,z_size,conv_dim = 32):
        super(Generator,self).__init__()
        self.conv_dim = conv_dim
        self.z = z_size
        
        self.conv0 = tconv(z_size,conv_dim * 32,4,1,0)
        self.conv1 = tconv(conv_dim * 32,conv_dim * 16,4)
        self.conv2 = tconv(conv_dim * 16,conv_dim * 8,4) # 256,16,16
        self.conv3 = tconv(conv_dim * 8,conv_dim *4,4) # 128,32,32
        self.conv4 = tconv(conv_dim * 4,conv_dim *2,4) # 64,64,64
        self.conv5 = nn.ConvTranspose2d(conv_dim *2,3,3,1,1) # 3,64,64
        
    def forward(self,x):
        x = x.view(-1,self.z,1,1)
        x = F.relu(self.conv0(x)) # 1024 * 4 * 4
        x = F.relu(self.conv1(x)) # 512 * 8 * 8
        #print(x.size())
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))# 256* 16*16
        x = F.relu(self.conv4(x))
        x = torch.tanh(self.conv5(x)) # 128
        
        return x
    

In [ ]:
def weights_init(m):
    """
    Takes as input a neural network m that will initialize all its weights.
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
# define hyperparams
CONV_DIM = 32
Z_SIZE = 100

# define discriminator and generator
D = Des(CONV_DIM)
#D.apply(weights_init)
G = Generator(Z_SIZE, CONV_DIM)
#G.apply(weights_init)
print(D)
print()
print(G)

train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    # move models to GPU
    G.cuda()
    D.cuda()
    print('GPU available for training. Models moved to GPU')
else:
    print('Training on CPU.')
    

Des 的loss

In [ ]:
def real_loss(D_out, smooth=False):
    batch_size = D_out.size(0)
    if smooth:
        labels = torch.ones(batch_size)*0.9
    else:
        labels = torch.ones(batch_size) # real labels = 1 
    if train_on_gpu:
        labels = labels.cuda()
    # binary cross entropy with logits loss
    criterion = nn.BCELoss()
    # calculate loss
    loss = criterion(D_out.squeeze(), labels)
    return loss

def fake_loss(D_out):
    batch_size = D_out.size(0)
    labels = Variable(torch.zeros(batch_size)) # fake labels = 0
    if train_on_gpu:
        labels = labels.cuda()
    criterion = nn.BCELoss()
    # calculate loss
    loss = criterion(D_out.squeeze(), labels)
    return loss



In [ ]:
LR_G = 0.002
LR_D = 0.0002
beta1= 0.5
beta2= 0.999

# Create optimizers for the discriminator and generator
d_optimizer = optim.Adam(D.parameters(), lr = LR_D,betas=[beta1, beta2])
g_optimizer = optim.Adam(G.parameters(), lr = LR_G,betas=[beta1, beta2])

In [ ]:
import pickle as pkl

# training hyperparams
num_epochs = 100

# keep track of loss and generated, "fake" samples
samples = []
losses = []

print_every = 300

# Get some fixed data for sampling. These are images that are held
# constant throughout training, and allow us to inspect the model's performance
sample_size=16
fixed_z = np.random.uniform(-1, 1, size=(sample_size, Z_SIZE))
fixed_z = torch.from_numpy(fixed_z).float()

# train the network
for epoch in tqdm(range(num_epochs)):
    for batch_i, (real_images, _) in enumerate(trainloader):     
        batch_size = real_images.size(0)
        # ============================================
        #            TRAIN THE DISCRIMINATOR
        # ============================================
        d_optimizer.zero_grad()
        
        # 1. Train with real images
        # Compute the discriminator losses on real images 
        if train_on_gpu:
            real_images = real_images.cuda()
        D_real = D(real_images)
        d_real_loss = real_loss(D_real,smooth=True)
        d_real_loss.backward()
        D_real = D_real.mean().item()
        
        
        # 2. Train with fake images
        # Generate fake images
        z = np.random.uniform(-1, 1, size=(batch_size, Z_SIZE))
        z = torch.from_numpy(z).float()
        # move x to GPU, if available
        if train_on_gpu:
            z = z.cuda()
            
        fake_images = G(z)
        # Compute the discriminator losses on fake images            
        D_fake = D(fake_images.detach())
        d_fake_loss = fake_loss(D_fake)
        d_fake_loss.backward()
        D_fake_1 = D_fake.mean().item()
        # add up loss and perform backprop
        d_loss = d_real_loss + d_fake_loss
        d_optimizer.step()
        
        # =========================================
        #            TRAIN THE GENERATOR
        # =========================================
        g_optimizer.zero_grad()
        
        # 1. Train with fake images and flipped labels
        # Compute the discriminator losses on fake images 
        D_fake = D(fake_images)
        g_loss = real_loss(D_fake,smooth=True) # use real loss to flip labels
        D_fake_2 = D_fake.mean().item()
        # perform backprop
        g_loss.backward()
        g_optimizer.step()

        # Print some loss stats
        if batch_i % print_every == 0:
            # append discriminator loss and generator loss
            losses.append((d_loss.item(), g_loss.item()))
            # print discriminator and generator loss
            print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}| Dreal_loss: {:6.4f}| Dfake1_loss: {:6.4f}| Dfake2_loss: {:6.4f}'.format(
                    epoch+1, num_epochs, d_loss.item(), g_loss.item(), D_real,D_fake_1,D_fake_2))

    
    ## AFTER EACH EPOCH##    
    # generate and save sample, fake images
    G.eval() # for generating samples
    if train_on_gpu:
        fixed_z = fixed_z.cuda()
    samples_z = G(fixed_z)
    samples.append(samples_z)
    G.train() # back to training mode


# Save training generator samples
with open('train_samples.pkl', 'wb') as f:
    pkl.dump(samples, f)

In [ ]:
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator', alpha=0.5)
plt.plot(losses.T[1], label='Generator', alpha=0.5)
plt.title("Training Losses")
plt.legend()

In [ ]:
samples[0].size()

In [ ]:
def view_samples(epoch, samples):
    fig, axes = plt.subplots(figsize=(16,4), nrows=2, ncols=8, sharey=True, sharex=True)
    for ax, img in zip(axes.flatten(), samples[epoch]):
        img = img.detach().cpu().numpy()
        img = np.transpose(img, (1, 2, 0))
        
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        im = ax.imshow(img.reshape((64,64,3)))

_ = view_samples(-1, samples)

In [ ]:
if not os.path.exists('../output_images'):
    os.mkdir('../output_images')
    
im_batch_size = 50
n_images=10000

for i_batch in tqdm(range(0, n_images, im_batch_size)):
    fixed_z = np.random.uniform(-1, 1, size=(im_batch_size, Z_SIZE))
    fixed_z = torch.from_numpy(fixed_z).float()
    if train_on_gpu:
        fixed_z = fixed_z.cuda()
    gen_images = G(fixed_z)
    images = gen_images.to("cpu").clone().detach()
    images = images.numpy().transpose(0, 2, 3, 1)
    for i_image in range(gen_images.size(0)):
        save_image(gen_images[i_image, :, :, :], os.path.join('../output_images', f'image_{i_batch+i_image:05d}.png'))

In [ ]:
import shutil
shutil.make_archive('images', 'zip', '../output_images')
